<a href="https://colab.research.google.com/github/almond5/CAP5510_Final_Project/blob/main/qna_method/qna_gpt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Packages***

In [1]:
%pip install transformers
%pip install sacremoses
%pip install torch
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## ***GPT-2***

In [2]:
import torch
from transformers import AutoTokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialogRPT-updown")
model = GPT2ForSequenceClassification.from_pretrained("microsoft/DialogRPT-updown", num_labels=3, ignore_mismatched_sizes=True)
ds = load_dataset("qiaojin/PubMedQA", "pqa_labeled")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/812 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at microsoft/DialogRPT-updown and are newly initialized because the shapes did not match:
- score.weight: found shape torch.Size([1, 1024]) in the checkpoint and torch.Size([3, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## ***200 Samples, 5 Epochs***

### **Trial 1**

In [3]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(200))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 100
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-05f89e883215>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,1.028893
2,No log,1.020071
3,No log,1.179647
4,No log,1.353337
5,1.103700,1.402632


Test set evaluation: {'eval_loss': 1.0937933921813965, 'eval_runtime': 5.1753, 'eval_samples_per_second': 9.661, 'eval_steps_per_second': 9.661, 'epoch': 5.0}


In [4]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")


0
Question: Is the combination with 2-methoxyestradiol able to reduce the dosages of chemotherapeutices in the treatment of human ovarian cancer?
Context: {'contexts': ['The endogenous estradiol metabolite, 2-methoxyestradiol (2ME), has been shown to be a potent inhibitor of cell growth and a strong anti-angiogenic substance. We investigated for the first time whether in vitro combinations of 2ME with various chemotherapeutic compounds may result in an additive inhibitory effect on the proliferation of human ovary cancer cells.', 'As a model two different human ovary cancer cell lines were used. All cell lines were incubated with equimolar concentrations of 2ME (0.8-25 microM) and the chemotherapeutics epirubicine, doxorubicine, paclitaxel, docetaxel, carboplatin, vinorelbine, 5-fluorouracil and mafosfamide. Proliferation was measured after four days using the ATP-chemosensitivity test.', 'For both ovary cancer cell lines a significant additive effect of 2ME with epirubicine and carbop

### **Trial 2**

In [5]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(200))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 100
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-05f89e883215>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.198342
2,No log,1.433033
3,No log,1.482104
4,No log,1.632276
5,1.369800,1.548377


Test set evaluation: {'eval_loss': 1.3204588890075684, 'eval_runtime': 5.2135, 'eval_samples_per_second': 9.59, 'eval_steps_per_second': 9.59, 'epoch': 5.0}


In [6]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")

0
Question: Can bisphosphonate treatment be stopped in a growing child with skeletal fragility?
Context: {'contexts': ['Cyclical pamidronate therapy in a 2-year-old child with skeletal fragility resulted in remodelling of vertebral fractures and improvement in bone mineral density (BMD) at distal radial and spinal sites. The BMD at both sites decreased precipitously within 24 months of stopping treatment, raising the question as to whether bisphosphonates can be stopped in a growing child with skeletal fragility.', 'At age 23 months, a male toddler sustained a low trauma fracture of his right femur. Skeletal radiographs revealed generalised osteopenia with multiple vertebral body fractures. He was diagnosed with type IV osteogenesis imperfecta; however, no mutations were found in COL1A1 or COL1A2 genes.', 'This case report presents bone densitometry data before, during and after bisphosphonate treatment. Axial QCT was main outcome from 2 years of age; DXA and pQCT were taken after age 

### **Trial 3**

In [7]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(200))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 100
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-05f89e883215>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.339390
2,No log,1.400059
3,No log,1.695157
4,No log,1.505449
5,1.270100,1.378086


Test set evaluation: {'eval_loss': 1.4545308351516724, 'eval_runtime': 5.2704, 'eval_samples_per_second': 9.487, 'eval_steps_per_second': 9.487, 'epoch': 5.0}


In [8]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")

0
Question: Can bisphosphonate treatment be stopped in a growing child with skeletal fragility?
Context: {'contexts': ['Cyclical pamidronate therapy in a 2-year-old child with skeletal fragility resulted in remodelling of vertebral fractures and improvement in bone mineral density (BMD) at distal radial and spinal sites. The BMD at both sites decreased precipitously within 24 months of stopping treatment, raising the question as to whether bisphosphonates can be stopped in a growing child with skeletal fragility.', 'At age 23 months, a male toddler sustained a low trauma fracture of his right femur. Skeletal radiographs revealed generalised osteopenia with multiple vertebral body fractures. He was diagnosed with type IV osteogenesis imperfecta; however, no mutations were found in COL1A1 or COL1A2 genes.', 'This case report presents bone densitometry data before, during and after bisphosphonate treatment. Axial QCT was main outcome from 2 years of age; DXA and pQCT were taken after age 

## ***300 samples; 5 epochs***

### **Trial 1**

In [9]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(300))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 200
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-6c1aedc3bee5>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.956585
2,No log,1.482231
3,1.436700,1.137594
4,1.436700,1.472975
5,1.336500,1.299560


Test set evaluation: {'eval_loss': 1.4093353748321533, 'eval_runtime': 5.213, 'eval_samples_per_second': 9.591, 'eval_steps_per_second': 9.591, 'epoch': 5.0}


In [10]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")


0
Question: Is the use of cyanoacrylate in intestinal anastomosis a good and reliable alternative?
Context: {'contexts': ['The present study aims to compare strength, healing, and operation time of experimental intestinal anastomoses performed by polyglactin 910 (Vicryl; Ethicon, Edinburgh, United Kingdom) sutures with ethyl-2-cyanoacrylate glue (Pattex; Henkel, Dusseldorf, Germany).', "Ninety-six Sprague-Dawley rats were divided into 2 (groups E and L). Each group was further subdivided into 6 subgroups (EA1, EA2, EA3, EB1, EB2, EB3, LA1, LA2, LA3, LB1, LB2, LB3), each containing 8 rats. Intestinal anastomosis was performed by polyglactin 910 sutures in A subgroups and with ethyl-2-cyanoacrylate in B subgroups. The anastomosis was end to end in A1 and B1, side to side in A2 and B2, and end to side in A3 and B3. Time for anastomosis performance (AT) was recorded. In group E, bursting pressures and hydroxyproline levels were determined on the second postoperative day, whereas in group L

### **Trial 2**

In [11]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(400))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 200
val_size = 50
test_size = 150

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-f21029c263a5>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.506206
2,No log,1.658632
3,1.327000,1.412054
4,1.327000,1.459592
5,1.301700,1.350691


Test set evaluation: {'eval_loss': 1.496770977973938, 'eval_runtime': 15.81, 'eval_samples_per_second': 9.488, 'eval_steps_per_second': 9.488, 'epoch': 5.0}


In [12]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")


0
Question: Convulsions and retinal haemorrhage: should we look further?
Context: {'contexts': ['The prevalence of retinal haemorrhages after convulsions is not well established. As these haemorrhages are considered characteristic of child abuse, we investigated their occurrence after convulsive episodes to see whether the finding of haemorrhage should prompt further investigation.', 'Prospective study of 153 children (aged 2 months to 2 years), seen in the emergency department after a convulsive episode. After a thorough history and physical examination, a retinal examination was performed by an ophthalmologist. If findings were positive, further investigation was undertaken to rule out systemic disorder or child abuse.', 'One child was found with unilateral retinal haemorrhages following an episode of a simple febrile convulsion. A thorough investigation uncovered no other reason for this finding.'], 'labels': ['BACKGROUND AND AIMS', 'METHODS', 'RESULTS'], 'meshes': ['Child Abuse', '

### **Trial 3**

In [13]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(300))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 200
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-6c1aedc3bee5>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.695105
2,No log,2.060489
3,1.084100,1.602969
4,1.084100,2.456620
5,1.172500,2.298772


Test set evaluation: {'eval_loss': 2.1539063453674316, 'eval_runtime': 5.2599, 'eval_samples_per_second': 9.506, 'eval_steps_per_second': 9.506, 'epoch': 5.0}


In [14]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")

0
Question: Is the use of cyanoacrylate in intestinal anastomosis a good and reliable alternative?
Context: {'contexts': ['The present study aims to compare strength, healing, and operation time of experimental intestinal anastomoses performed by polyglactin 910 (Vicryl; Ethicon, Edinburgh, United Kingdom) sutures with ethyl-2-cyanoacrylate glue (Pattex; Henkel, Dusseldorf, Germany).', "Ninety-six Sprague-Dawley rats were divided into 2 (groups E and L). Each group was further subdivided into 6 subgroups (EA1, EA2, EA3, EB1, EB2, EB3, LA1, LA2, LA3, LB1, LB2, LB3), each containing 8 rats. Intestinal anastomosis was performed by polyglactin 910 sutures in A subgroups and with ethyl-2-cyanoacrylate in B subgroups. The anastomosis was end to end in A1 and B1, side to side in A2 and B2, and end to side in A3 and B3. Time for anastomosis performance (AT) was recorded. In group E, bursting pressures and hydroxyproline levels were determined on the second postoperative day, whereas in group L

## **400 Samples 5 Epochs**

### **Trial 1**

In [15]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(400))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 300
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-6cd25f7cb1d0>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.138530
2,1.560900,1.452946
3,1.560900,1.748201
4,1.532100,1.791056
5,1.245100,1.525807


Test set evaluation: {'eval_loss': 2.9139959812164307, 'eval_runtime': 5.3483, 'eval_samples_per_second': 9.349, 'eval_steps_per_second': 9.349, 'epoch': 5.0}


In [16]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")

1
Question: Do general practice characteristics influence uptake of an information technology (IT) innovation in primary care?
Context: {'contexts': ['Recent evaluations of IT innovations in primary care have highlighted variations between centres and practices in uptake and use. We evaluated whether structural characteristics of a general practice were associated with variations in use of a web-based clinical information system underpinning a Managed Clinical Network in diabetes, between the years 2001 and 2003.', 'Using a computerised audit trail, we calculated the numbers of web-based operations that occurred in each practice, stratified by staff type and year, and adjusted for the numbers of registered diabetic patients. In regression analyses, we determined whether total use was associated with structural characteristics of the practice (total list size, training status, numbers of GPs (general practitioners), mean age of the GPs, numbers of female GPs, level of deprivation of the

### **Trial 2**

In [17]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(400))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 300
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-6cd25f7cb1d0>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.665722
2,1.139000,1.851719
3,1.139000,1.856004
4,1.090100,1.769300
5,1.064500,1.693997


Test set evaluation: {'eval_loss': 3.076169490814209, 'eval_runtime': 5.1638, 'eval_samples_per_second': 9.683, 'eval_steps_per_second': 9.683, 'epoch': 5.0}


In [18]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")

1
Question: Do general practice characteristics influence uptake of an information technology (IT) innovation in primary care?
Context: {'contexts': ['Recent evaluations of IT innovations in primary care have highlighted variations between centres and practices in uptake and use. We evaluated whether structural characteristics of a general practice were associated with variations in use of a web-based clinical information system underpinning a Managed Clinical Network in diabetes, between the years 2001 and 2003.', 'Using a computerised audit trail, we calculated the numbers of web-based operations that occurred in each practice, stratified by staff type and year, and adjusted for the numbers of registered diabetic patients. In regression analyses, we determined whether total use was associated with structural characteristics of the practice (total list size, training status, numbers of GPs (general practitioners), mean age of the GPs, numbers of female GPs, level of deprivation of the

### **Trial 3**

In [19]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "uncertain": 2, "maybe": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(400))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

train_size = 300
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-19-6cd25f7cb1d0>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.500147
2,0.814200,1.902829
3,0.814200,1.794714
4,0.878800,1.978878
5,0.820300,1.832597


Test set evaluation: {'eval_loss': 3.2277822494506836, 'eval_runtime': 5.1616, 'eval_samples_per_second': 9.687, 'eval_steps_per_second': 9.687, 'epoch': 5.0}


In [20]:
# Define the function to predict using the trained model
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(predicted_class)
    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain", 2: "maybe"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if true_answer.lower() == predicted_answer:
        num_correct += 1

    print("="*80)

# Print accuracy
print(f"Accuracy: {num_correct / len(test_dataset)}")

1
Question: Do general practice characteristics influence uptake of an information technology (IT) innovation in primary care?
Context: {'contexts': ['Recent evaluations of IT innovations in primary care have highlighted variations between centres and practices in uptake and use. We evaluated whether structural characteristics of a general practice were associated with variations in use of a web-based clinical information system underpinning a Managed Clinical Network in diabetes, between the years 2001 and 2003.', 'Using a computerised audit trail, we calculated the numbers of web-based operations that occurred in each practice, stratified by staff type and year, and adjusted for the numbers of registered diabetic patients. In regression analyses, we determined whether total use was associated with structural characteristics of the practice (total list size, training status, numbers of GPs (general practitioners), mean age of the GPs, numbers of female GPs, level of deprivation of the